# I. Rigideces de Entrepiso
# Fórmulas de Wilbur-Biggs

## 1. Librerías

In [ ]:
import numpy as np

from matplotlib import pyplot as plt

## 2. Geometría de Pórtico

### 2.1 Número de Pisos

In [ ]:
n_pisos = int(input('* Ingrese el número de pisos: '))

print(f'* El pórtico tiene {n_pisos} plantas.')


### 2.2 Elevaciones de Entrepisos

In [ ]:
h = np.zeros(n_pisos)
    
for i in range(n_pisos):
    h[i] = float(input(f'* Elevación del entrepiso {i + 1} (m): '))

for i in range(n_pisos):
    print(f'\n* Elevación H-{n_pisos - i}: {h[n_pisos - i - 1]} m')

### 2.3 Columnas y Vigas

In [ ]:
# Número de columnas
n_col = int(input('* Indique el número de columnas: '))

# Número de vigas por piso
n_vigas = n_col - 1

print(f'* El pórtico es de {n_col} columnas y ', end = '')

if n_vigas > 1:
    print(f'{n_vigas} vigas por piso.')
else:
    print(f'{n_vigas} viga por piso.')


### 2.4 Tipo de Apoyo en la Base

In [ ]:
while True:
    t_apoyo = input('* Tipo de apoyo ([E]mpotrado o [A]rticulado): ')
        
    if t_apoyo.upper() == 'E':
        print('* El pórtico es empotrado en su base.')
        apoyo_articulado = False
        break
    elif t_apoyo.upper() == 'A':
        print('* El pórtico es articulado en su base.')
        apoyo_articulado = True
        break
    else:
        print('! Ingrese un tipo de apoyo válido.')


### 2.5 Modelo Matemático

In [ ]:
plt.rcParams['figure.figsize'] = 6, 8
plt.rcParams['font.family'] = 'Cascadia Mono'
plt.style.use('grayscale')

In [ ]:
ax = plt.axes()
ax.set_title('Modelo de Pórtico', fontsize = '25', pad = 20)

ax.grid(False)
ax.axes.xaxis.set_visible(False)

# Trazado de vigas
H = 0
B = 2 * sum(h) / 3

for i in range(n_pisos):
    H += h[i]
    plt.hlines(H, 0, B, linewidth = 3, color = '#424242')

# Trazado de columnas
d = 0

for i in range(n_col):
    plt.vlines(d, 0, H,  linewidth = 3, color = '#424242')
    d += B / (n_col - 1)

# Ubicación de apoyos
if apoyo_articulado:
    d = 0
    for i in range(n_col):
        plt.plot(d, 0, marker = 10, markersize = 16, color = '#424242')
        d += B / (n_col - 1)

ax.axhline(y = 0, linewidth = 2, color = '#8C8C8C')

ax.set_xlim(-B / 4, 5 * B / 4)

plt.show()


## 3. Cálculo de Rigideces

### 3.1 Modulo de Elasticidad

In [ ]:
E = float(input('* Modulo de elasticidad (Pa): '))
print(f'* Módulo de elasticidad [E]: {E}')


### 3.2 Fórmulas de Wilbur-Biggs

In [ ]:
## Fórmulas de primer piso

# Apoyos articulados
def Kp1_articulado(Kc1, Kv1, h1, h2, E):
    K = 24 * E
    K /= h1
    K /= 8 * h1 / Kc1 + (2 * h1 + h2) / Kv1
    return K

# Apoyos empotrados
def Kp1_empotrado(Kc1, Kv1, h1, E):
    K = 24 * E
    K /= np.power(h1, 2)
    K /= 2 / Kc1 + 1 / (Kv1 + Kc1 / 12)
    return K

$Apoyos \ articulados$

$$
K_i = \frac{24 \ E}{H_i^2 \left[\displaystyle \frac{2}{\sum k_{c_i}}+ \displaystyle \frac{1}{\sum k_{v_i} + \displaystyle  \frac{\sum k_{c_i}}{12}}\right]}
$$

$Apoyos \ empotrados$

$$
K_i = \frac{24 \ E}{H_i \left[\displaystyle  \frac{8H_i}{\sum k_{c_i}} + \frac{2H_i + H_{i+1}}{\sum k_{v_i}}\right]}
$$

In [ ]:
# Fórmula de segundo piso con
# un primer piso articulado

def Kp2_articulado(Kc1, Kv0, Kv1, h0, h1, h2, E):
    K = 48 * E
    K /= h1
    K /= (4 * h1 / Kc1) + ((h1 + h2) / Kv1) + ((2 * h0 + h1) / Kv0)
    return K

$Segundo \ piso \ con \ primer \ piso \ articulado$

$$
K_i = \frac{48 \ E}{H_i \left[\displaystyle  \frac{4H_i}{\sum k_{c_i}} + \frac{H_i + H_{i+1}}{\sum k_{v_i}} + \frac{2 H_{i-1} + H_{i}}{\sum k_{v_{i-1}}}\right]}
$$



In [ ]:
# Fórmula general de entrepiso

def Kp_entrepiso(Kc1, Kv0, Kv1, h0, h1, h2, E):
    K = 48 * E
    K /= h1
    K /= (4 * h1 / Kc1) + ((h0 + h1) / Kv0) + ((h1 + h2) / Kv1)
    return K


$Fórmula \ general \ de \ entrepiso$

$$
K_i = \frac{48 \ E}{H_i \left[\displaystyle  \frac{4H_i}{\sum k_{c_i}} + \frac{H_{i-1} + H_{i}}{\sum k_{v_{i-1}}} + \frac{ H_{i} + H_{i+1}}{\sum k_{v_{i}}}\right]}
$$

### 3.2 Rigideces de Columnas

In [ ]:
Kc = np.empty((n_pisos, n_col))

for i in range(n_pisos):
    print(f'- Piso {i + 1}')
    for j in range(n_col):
        Kc[i, j] = float(input(f'* Piso {i + 1} - Columna {j + 1} [Kc]: '))
        print(f'\t* Columna {j + 1} (Kc): {Kc[i, j]}')

        if j == n_col - 1:
            print(f'\t\t> Kc[Piso{i + 1}] = {sum(Kc[i,:])}\n')


### 3.3 Rigideces de Vigas

In [ ]:
Kv = np.empty((n_pisos, n_vigas))

for i in range(n_pisos):
    print(f'- Piso {i + 1}')
    for j in range(n_vigas):
        Kv[i, j] = float(input(f'* Piso {i + 1} - Viga {j + 1} [Kv]: '))
        print(f'\t* Viga {j + 1} (Kv): {Kv[i, j]}')

        if j == n_vigas - 1:
            print(f'\t\t> Kv[Piso{i + 1}] = {sum(Kv[i,:])}\n')


### 3.4 Rigideces de Entrepisos

In [ ]:
K = np.empty(n_pisos)

for i in range(n_pisos):
    ### Rigideces de elementos ###

    ## Columnas ##

    # Piso actual
    Kc1 = sum(Kc[i,:])

    # Piso superior
    try: 
        Kc2 = sum(Kc[i + 1,:])
    except IndexError:
        Kc2 = 0

    # Piso inferior
    try: 
        Kc0 = sum(Kc[i - 1,:])
    except IndexError:
        Kc0 = 0

    ## Vigas ##

    # Piso actual
    Kv1 = sum(Kc[i,:])

    # Piso superior
    try: 
        Kv2 = sum(Kc[i + 1,:])
    except IndexError:
        Kv2 = 0

    # Piso inferior
    try: 
        Kv0 = sum(Kc[i - 1,:])
    except IndexError:
        Kv0 = 0


    ### Elevaciones ###
    
    # Piso actual
    h1 = h[i]

    # Piso superior
    try:
        h2 = h[i + 1]
    except IndexError:
        h2 = 0

    # Piso inferior
    try:
        h0 = h[i - 1]
    except IndexError:
        h0 = 0


    ### Cálculos ###

    if i == 0: # Primer piso
        if apoyo_articulado:
            np.append(K, Kp1_articulado(Kc1, Kv1, h1, h2, E))
        else:
            np.append(K, Kp1_empotrado(Kc1, Kv1, h1, E))

    elif i == 1: # Segundo piso
        if apoyo_articulado:
            np.append(K, Kp2_articulado(Kc1, Kv0, Kv1, h0, h1, h2, E))
        else:
            np.append(K, Kp_entrepiso(Kc1, Kv0, Kv1, h0, h1, h2, E))
    
    else:
        np.append(K, Kp_entrepiso(Kc1, Kv0, Kv1, h0, h1, h2, E))

    print(f'* Entrepiso {i + 1}')
    print(f'\t> K[{i + 1}] = {K[i]}\n')
